In [ ]:
#Rafael Vilela Santa Rosa
!pip install -q pvlib
import time
import pvlib
from pvlib import location, irradiance, tools
from pvlib.iotools import read_tmy3
import pandas as pd
from matplotlib import pyplot as plt
import pathlib
import numpy as np
from scipy.optimize import curve_fit
from scipy.signal import hilbert
from scipy.stats import skew, kurtosis, variation

tempo_inicial=(time.time())

#obs: arquivo csv escolhido não considera um ano bissexto

energia_hs = [70, 82.4, 122.9, 150.4, 166, 176.9, 174.9, 162.4, 123.8, 102.4, 68.2, 67.5]

df1 = pd.DataFrame({'energia_hs': energia_hs})


# get full path to the data directory
DATA_DIR = pathlib.Path(pvlib.__file__).parent / 'data'

# get TMY3 dataset
#tmy, metadata = read_tmy3(DATA_DIR / '703165TY.csv', coerce_year=2023)
tmy, metadata = read_tmy3(DATA_DIR / '723170TYA.CSV', coerce_year=2023)

# create location object to store lat, lon, timezone
location = location.Location.from_tmy(metadata)

# print metadata
for key, value in metadata.items():
    print(f'{key}: {value}')
latitude = metadata['latitude']
Φ = latitude

# calculate the necessary variables to do transposition.
times = pd.date_range(start='2023-01-01', end='2023-12-31 23:59:59', freq='1H')

# create a new DataFrame that only contains the necessary columns and that has a consistent time index
df = pd.DataFrame(index=times)
df['DHI'] = tmy['DHI'].values
df['DNI'] = tmy['DNI'].values
df['GHI'] = tmy['GHI'].values

max_value1 = df['DHI'].max()
print(f"Valor máximo DHI: {max_value1}")
max_value2 = df['DNI'].max()
print(f"Valor máximo DNI: {max_value2}")
max_value3 = df['GHI'].max()
print(f"Valor máximo GHI: {max_value3}")


solar_position = location.get_solarposition(times)

# calculate extra terrestrial radiation
dni_extra = pvlib.irradiance.get_extra_radiation(df.index)


Optimal_tilt_angle_NH = 1.3793 + Φ*(1.2011 + Φ*(-0.014404 + Φ*0.000080509))
print(f'Inclinação ótima:{Optimal_tilt_angle_NH}')



############################################### inicio


############################################### beam

def aoi_projection(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth):


    projection = (
        tools.cosd(surface_tilt) * tools.cosd(solar_zenith) +
        tools.sind(surface_tilt) * tools.sind(solar_zenith) *
        tools.cosd(solar_azimuth - surface_azimuth))

    # GH 1185
    projection = np.clip(projection, -1, 1)

    try:
        projection.name = 'aoi_projection'
    except AttributeError:
        pass

    return projection

def beam_component(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
                   dni):

    beam = dni * aoi_projection(surface_tilt, surface_azimuth,
                                solar_zenith, solar_azimuth)
    beam = np.maximum(beam, 0)

    return beam

df['beam']=beam_component(surface_tilt=30, surface_azimuth=90, solar_zenith=solar_position['apparent_zenith'], solar_azimuth=solar_position['azimuth'],
                   dni=df['DNI'])

############################################### ground 

SURFACE_ALBEDOS = {'urban': 0.18,
                   'grass': 0.20,
                   'fresh grass': 0.26,
                   'soil': 0.17,
                   'sand': 0.40,
                   'snow': 0.65,
                   'fresh snow': 0.75,
                   'asphalt': 0.12,
                   'concrete': 0.30,
                   'aluminum': 0.85,
                   'copper': 0.74,
                   'fresh steel': 0.35,
                   'dirty steel': 0.08,
                   'sea': 0.06}

df['ground_reflected']= pvlib.irradiance.get_ground_diffuse(surface_tilt=30, ghi=df['GHI'], albedo=0.2, surface_type=None)


############################################### diffuse

# Hay Davies
df['haydavies_diffuse'] = pvlib.irradiance.haydavies(surface_tilt=30, surface_azimuth=270,
                                        dhi=df['DHI'], dni=df['DNI'], dni_extra=dni_extra,
                                        solar_zenith=solar_position['apparent_zenith'],
                                        solar_azimuth=solar_position['azimuth'])


print("\n-----------Dados-----------")


beam_zeros = df['beam'][df['beam'] == 0].count()
print(f"Number of zeros in beam model: {beam_zeros}")

beam_high = df['beam'][df['beam'] > 1367].count()
print(f"Number of high numbers (>1367) in beam model: {beam_high}")

# plot the results
df['beam'].plot()
plt.ylabel('Beam Irradiance [W/m^2]')
plt.title('Beam Model')
plt.show()

ground_zeros = df['ground_reflected'][df['ground_reflected'] == 0].count()
print(f"Number of zeros in GR: {ground_zeros}")

# plot the results
df['ground_reflected'].plot()
plt.ylabel('Ground Reflected Irradiance [W/m^2]')
plt.title('Ground Reflected Model')
plt.show()

haydavies_diffuse_zeros = df['haydavies_diffuse'][df['haydavies_diffuse'] == 0].count()
print(f"Number of zeros in haydavies_diffuse model: {haydavies_diffuse_zeros}")

# plot the results
df['haydavies_diffuse'].plot()
plt.ylabel('Diffuse Sky Irradiance [W/m^2]')
plt.title('Diffuse Sky Irradiance using Hay Davies Model')
plt.show()

############################################### somatorio

print("\n-----------Dados agrupados-----------")

df['total_irradiance'] = df['haydavies_diffuse'] + df['ground_reflected'] + df['beam']


skewness_list = []
envelope_list = []
kurtosis_list = []
cv_list = []
fs=1

# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Get the data for the current day
    day_data = df.loc[day.strftime('%Y-%m-%d'), 'total_irradiance']
    
    # Calculate Hilbert transform of the data
    analytic_signal = hilbert(day_data)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * fs)
    
    # Calculate skewness and append to list
    skewness_list.append(skew(day_data))
    
    # Calculate mean envelope and append to list
    envelope_list.append(np.mean(amplitude_envelope))
    
    # Calculate kurtosis and append to list
    kurtosis_list.append(kurtosis(day_data))
    
    # Calculate coefficient of variation and append to list
    cv_list.append(variation(day_data))
    
mean_skewness = np.mean(skewness_list)
mean_envelope = np.mean(envelope_list)
mean_kurtosis = np.mean(kurtosis_list)
mean_cv = np.mean(cv_list)

print(f"Mean skewness: {mean_skewness}")
print(f"Mean envelope: {mean_envelope}")
print(f"Mean kurtosis: {mean_kurtosis}")
print(f"Mean Coefficient of Variation: {mean_cv}")

# Calculate quartiles
Q1 = df['total_irradiance'].quantile(0.25)
Q2 = df['total_irradiance'].quantile(0.5)
Q3 = df['total_irradiance'].quantile(0.75)
Q4 = df['total_irradiance'].quantile(1)
P95 = df['total_irradiance'].quantile(0.95)
P98 = df['total_irradiance'].quantile(0.98)
P99 = df['total_irradiance'].quantile(0.99)

print("First Quartile:", Q1)
print("Second Quartile (Median):", Q2)
print("Third Quartile:", Q3)
print("Fourth Quartile (Max):", Q4)
print("95th Percentile:", P95)
print("98th Percentile:", P98)
print("99th Percentile:", P99)
print("\n")

total_irradiance_zeros = df['total_irradiance'][df['total_irradiance'] == 0].count()
print(f"Number of zeros in total_irradiance: {total_irradiance_zeros}")

haydavies_high = df['total_irradiance'][df['total_irradiance'] > 1367].count()
print(f"Number of high numbers in haydavies model: {haydavies_high}")

haydavies_mid = df['total_irradiance'][df['total_irradiance'] > P98].count()
print(f"Number of mid numbers (>P99) in haydavies model: {haydavies_mid}")

############################################### valores sem tratamento de outlier

# Create a new column for the month
df['month'] = df.index.month

# Group by month and sum the irradiance to get the total irradiance for each month
monthly_irradiance = df.groupby('month')['total_irradiance'].sum()

# The total irradiance is in units of W/m^2. 
monthly_energy = monthly_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
monthly_energy_kwh = monthly_energy / (1000 * 3600)


# Calcula a diferença percentual

percentual1=((monthly_energy_kwh.values/ df1['energia_hs'].values) * 100)

df2 = pd.DataFrame({'percentual': percentual1})

print('valor médio HS:')
print(df2['percentual'].mean())
print('\nPercentual HelioScope:')
print(df2['percentual'])

############################################### continuação


# plot the results
df['total_irradiance'].plot()
plt.ylabel('POA Irradiance [W/m^2]')
plt.title('POA Irradiance using Hay Davies Model')
plt.show()

# Para um mês específico:
specific_month = df.loc['2023-06']

specific_month['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()

# Para um dia específico:
specific_day = df.loc['2023-06-01']

specific_day['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()



############################################### correção inicial de outlier, se necessário

#df['total_irradiance'] = df['total_irradiance'].clip(upper=P98)

max_value = df['total_irradiance'].max()
print(f"Valor máximo: {max_value}")

############################################### identificação de outlier

"""

# Create a new column for the day
df['day'] = df.index.dayofyear

# Group by day and sum the irradiance to get the total irradiance for each day
daily_irradiance = df.groupby('day')['total_irradiance'].sum()

# The total irradiance is in units of W/m^2. To convert this to energy, we multiply by the number of seconds in an hour (3600),
# since each row in the DataFrame represents an hour.
daily_energy = daily_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
daily_energy_kwh = daily_energy / (1000 * 3600)

"""

# Função para representar a forma senoidal
def sinusoidal(x, a, b, c, d):
    return a * np.sin(b * (x - np.radians(c))) + d

# Agregar os dados por dia
daily_irradiance = df['total_irradiance'].resample('D').sum()

# Obter o dia do ano para usar como variável independente no ajuste
day_of_year = daily_irradiance.index.dayofyear

# Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
# O período é ajustado para 2*pi/365 para refletir o ciclo anual
# A fase horizontal é ajustada para colocar o pico no meio do ano (dia 182)
initial_parameters = [daily_irradiance.max(), 2 * np.pi / 365, 182, daily_irradiance.min()]

# Ajustar a função aos dados
parameters, _ = curve_fit(sinusoidal, day_of_year, daily_irradiance, p0=initial_parameters)

# Gerar um array de x para a função ajustada
x_fit = np.linspace(1, 365, 1000)

# Gerar os valores y da função ajustada
y_fit = sinusoidal(x_fit, *parameters)

print(f"A função senoidal de tendência é: {parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}")

# Plotar os dados originais e a função ajustada
plt.scatter(day_of_year, daily_irradiance, label='data')
plt.plot(x_fit, y_fit, color='red', label='fit')
plt.xlabel('Day of year')
plt.ylabel('Irradiação diária [Wh]')
plt.title('Sinusoidal fit to daily POA irradiance data')
plt.legend()
plt.text(170,500,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.show()



############################################### tratamento outlier 

print("\n-----------Tratamento de outlier-----------")


'''
# valores da função de tendência para cada dia do ano
trend_values = sinusoidal(day_of_year, *parameters)

# limite superior e inferior
upper_limit = trend_values * 1.20
lower_limit = trend_values * 1.05

# série pandas para os valores de tendência e limite superior/inferior para facilitar a manipulação
trend_series = pd.Series(trend_values, index=daily_irradiance.index)
upper_limit_series = pd.Series(upper_limit, index=daily_irradiance.index)
lower_limit_series = pd.Series(lower_limit, index=daily_irradiance.index)


##

# Resample the hourly data to daily data
daily_poa_global = df['total_irradiance'].resample('D').sum()

# Replace values in the daily data that are above the upper limit
daily_poa_global.where(daily_poa_global <= upper_limit_series, upper_limit_series, inplace=True)
daily_poa_global.where(daily_poa_global >= lower_limit_series, lower_limit_series, inplace=True)

# Resample the daily data back to hourly data
df['total_irradiance'] = daily_poa_global.resample('H').ffill() / 24

# plot the results
df['total_irradiance'].plot()
plt.ylabel('POA [W/m^2]')
plt.title('POA Irradiance using haydavies Model')
plt.show()

# Para um mês específico:
specific_month = df.loc['2023-06']

specific_month['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()

# Para um dia específico:
specific_day = df.loc['2023-06-01']

specific_day['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()

'''

############################################### tratamento outlier 2


# Create a new DataFrame to store the adjusted irradiance values
df_adjusted = df.copy()

daily_parameters = pd.DataFrame(columns=['a', 'b', 'c', 'd'], index=df.index.normalize().unique())

coef_upper = 1.20
coef_lower = 1.05

# Criação de um DataFrame para armazenar os parâmetros diários
daily_parameters = pd.DataFrame(index=pd.date_range('2023-01-01', '2023-12-31'), 
                                columns=['amplitude', 'period', 'phase_shift', 'vertical_shift'])


# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Obter os dados para o dia atual
    day_data = df.loc[day.strftime('%Y-%m-%d')]
    
    # Obter a hora do dia para usar como variável independente no ajuste
    hour_of_day = day_data.index.hour
    
    # Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
    # O período é ajustado para 2*pi para refletir o ciclo diário
    initial_parameters = [day_data['total_irradiance'].max(), 2 * np.pi, 12, day_data['total_irradiance'].min()]
    
    # Ajustar a função aos dados
    try:
        parameters, _ = curve_fit(sinusoidal, hour_of_day, day_data['total_irradiance'], p0=initial_parameters, maxfev=5000)
    except RuntimeError:
        print(f"Failed to fit curve for day {day}")
        continue
    # Armazenar os parâmetros no DataFrame
    daily_parameters.loc[day, ['amplitude', 'period', 'phase_shift', 'vertical_shift']] = parameters
    
# Loop por cada dia do ano novamente para ajustar os outliers
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Obter os dados para o dia atual
    day_data = df.loc[day.strftime('%Y-%m-%d')]
    
    # Obter a hora do dia para usar como variável independente no ajuste
    hour_of_day = day_data.index.hour
    
    # Obter os parâmetros para o dia atual
    parameters = daily_parameters.loc[day]
    
    # Calcular os valores da função de tendência para cada hora do dia
    trend_values = sinusoidal(hour_of_day, *parameters)
    
    # Definir um limite superior e inferior
    upper_limit = trend_values * coef_upper
    lower_limit = trend_values * coef_lower
    
    # série pandas para os valores de tendência e limite superior/inferior para facilitar a manipulação
    trend_series = pd.Series(trend_values, index=day_data.index)
    upper_limit_series = pd.Series(upper_limit, index=day_data.index)
    lower_limit_series = pd.Series(lower_limit, index=day_data.index)
    
    for i in range(len(day_data)):
        if day_data['total_irradiance'].iloc[i] > 0:                    
         # Substituir valores nos dados diários  
            if day_data['total_irradiance'].iloc[i] > upper_limit_series.iloc[i]:
                day_data['total_irradiance'].iloc[i] = upper_limit_series.iloc[i]
            elif day_data['total_irradiance'].iloc[i] < lower_limit_series.iloc[i]:
                day_data['total_irradiance'].iloc[i] = lower_limit_series.iloc[i]
    
             # Armazenar os dados ajustados de volta no DataFrame original
    df.loc[day.strftime('%Y-%m-%d'), 'total_irradiance'] = day_data['total_irradiance']

# df['total_irradiance'] ajustado para outliers com base na tendência senoidal diária

# para valores negativos por conta do ajuste da senoide
df['total_irradiance'] = df['total_irradiance'].clip(lower=0)

print("\n-----------Termino de tratamento de outlier-----------")


# plot the results
df['total_irradiance'].plot()
plt.ylabel('POA Irradiance [W/m^2]')
plt.title('POA Irradiance using Hay Davies Model')
plt.show()

# Para um mês específico:
specific_month = df.loc['2023-06']

specific_month['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()

# Para um dia específico:
specific_day = df.loc['2023-06-01']

specific_day['total_irradiance'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()

# um dia específico
specific_day = "2023-08-02"

# Obtenha a irradiação para esse dia
day_data = df.loc[specific_day]

# Obtenha os parâmetros para esse dia
parameters = daily_parameters.loc[specific_day]

# Gere a curva senoidal para esse dia
hour_of_day = np.arange(24)
sinusoidal_data = sinusoidal(hour_of_day, *parameters)

# Plote a irradiação e a curva senoidal
plt.figure(figsize=(10, 6))
plt.plot(day_data.index.hour, day_data['total_irradiance'], label='Irradiância')
plt.plot(hour_of_day, sinusoidal_data, label='Curva Senoidal', linestyle='--')
plt.xlabel('Hora do dia')
plt.ylabel('Irradiância [W/m²]')
plt.title(f'Irradiância e Curva Senoidal para o dia {specific_day}')
plt.legend()
plt.text(12,0,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.grid(True)
plt.show()

############################################### identificação outlier

# Função para representar a forma senoidal
def sinusoidal(x, a, b, c, d):
    return a * np.sin(b * (x - np.radians(c))) + d

# Agregar os dados por dia
daily_irradiance = df['total_irradiance'].resample('D').sum()

# Obter o dia do ano para usar como variável independente no ajuste
day_of_year = daily_irradiance.index.dayofyear

# Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
# O período é ajustado para 2*pi/365 para refletir o ciclo anual
# A fase horizontal é ajustada para colocar o pico no meio do ano (dia 182)
initial_parameters = [daily_irradiance.max(), 2 * np.pi / 365, 182, daily_irradiance.min()]

# Ajustar a função aos dados
parameters, _ = curve_fit(sinusoidal, day_of_year, daily_irradiance, p0=initial_parameters)

# Gerar um array de x para a função ajustada
x_fit = np.linspace(1, 365, 1000)

# Gerar os valores y da função ajustada
y_fit = sinusoidal(x_fit, *parameters)

print(f"A função senoidal de tendência é: {parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}")

# Plotar os dados originais e a função ajustada
plt.scatter(day_of_year, daily_irradiance, label='data')
plt.plot(x_fit, y_fit, color='red', label='fit')
plt.xlabel('Day of year')
plt.ylabel('Irradiação diária [Wh]')
plt.title('Sinusoidal fit to daily POA irradiance data - com tratamento de outliers')
plt.legend()
plt.text(170,500,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.show()


############################################### calculo de energia

# Create a new column for the month
df['month'] = df.index.month

# Group by month and sum the irradiance to get the total irradiance for each month
monthly_irradiance = df.groupby('month')['total_irradiance'].sum()

# The total irradiance is in units of W/m^2. 
monthly_energy = monthly_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
monthly_energy_kwh = monthly_energy / (1000 * 3600)

#monthly_energy_kwh

# Calcula a diferença percentual

percentual2=((monthly_energy_kwh.values/ df1['energia_hs'].values) * 100)

df3 = pd.DataFrame({'percentual': percentual2})

print('valor médio HS:')
print(df3['percentual'].mean())
print('\nPercentual HelioScope:')
print(df3['percentual'])


months = list(range(1, 13))
errors1 = np.abs(df2['percentual'] - 100)
errors2 = np.abs(df3['percentual'] - 100)

# Plot HelioScope data
plt.errorbar(months, df2['percentual'], yerr=errors1, fmt='o', color='blue', label='Original (em relação ao HelioScope)')

# Plot PVSyst data
plt.errorbar(months, df3['percentual'], yerr=errors2, fmt='o', color='red', label='Com correção de outlier (em relação ao HelioScope)')

# Adicione a linha em y=100
plt.axhline(100, color='black', linestyle='--')

# Adicione títulos e rótulos
plt.title('Comparação de percentuais mensais (Hay-Davies/HelioScope)')
plt.xlabel('Mês')
plt.ylabel('Percentual (%)')

# Adicione uma legenda
plt.legend()

for month, error1, error2 in zip(months, errors1, errors2):
    plt.text(month, df2['percentual'][month-1], f"{error1:.1f}%", ha='center', va='bottom')
    plt.text(month, df3['percentual'][month-1], f"{error2:.1f}%", ha='center', va='top')

# Exibir o gráfico
plt.show()

skewness_list2 = []
envelope_list2 = []
kurtosis_list2 = []
cv_list2 = []

# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Get the data for the current day
    day_data = df.loc[day.strftime('%Y-%m-%d'), 'total_irradiance']
    
    # Calculate Hilbert transform of the data
    analytic_signal = hilbert(day_data)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * fs)
    
    # Calculate skewness and append to list
    skewness_list2.append(skew(day_data))
    
    # Calculate mean envelope and append to list
    envelope_list2.append(np.mean(amplitude_envelope))
    
    # Calculate kurtosis and append to list
    kurtosis_list2.append(kurtosis(day_data))
    
    # Calculate coefficient of variation and append to list
    cv_list2.append(variation(day_data))
    
mean_skewness = np.mean(skewness_list2)
mean_envelope = np.mean(envelope_list2)
mean_kurtosis = np.mean(kurtosis_list2)
mean_cv = np.mean(cv_list2)

print(f"Mean skewness: {mean_skewness}")
print(f"Mean envelope: {mean_envelope}")
print(f"Mean kurtosis: {mean_kurtosis}")
print(f"Mean Coefficient of Variation: {mean_cv}")


tempo_final=(time.time())
tempo= tempo_final - tempo_inicial

print(f"{tempo} segundos")

print("Fim!")